In [2]:
import numpy as np
import pandas as pd
import json
import os

In [3]:
'''
A json post has two keys: data and comments. Comments also have the same elements as data, as well as 'children',
which are comments on the comment.
'''
# Eg) data
keys = {'_comments_by_id': {},
 '_fetched': False,
 'all_awardings': [],
 'allow_live_comments': False,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': 'shayalldamnday',
 'author_fullname': 't2_67rdsbe5',
 'author_premium': False,
 'awarders': [],
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': False,
 'can_mod_post': False,
 'category': None,
 'clicked': False,
 'comment_limit': 2048,
 'comment_sort': 'confidence',
 'content_categories': None,
 'contest_mode': False,
 'created': 1587166355.0,
 'created_utc': 1587137555.0,
 'discussion_type': None,
 'distinguished': None,
 'domain': 'self.depressed',
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'gildings': {},
 'hidden': False,
 'hide_score': False,
 'id': 'g348ft',
 'is_original_content': False,
 'is_video': False,
 'likes': None,
 'locked': False,
 'media': None,
 'media_embed': {},
 'media_only': False,
 'mod_note': None,
 'mod_reason_by': None,
 'mod_reason_title': None,
 'mod_reports': [],
 'name': 't3_g348ft',
 'no_follow': False,
 'num_comments': 31,
 'num_crossposts': 0,
 'num_reports': None,
 'over_18': False,
 'parent_whitelist_status': None,
 'permalink': '/r/depressed/comments/g348ft/i_hate_waking_up_in_the_mornings/',
 'pwls': None,
 'quarantine': False,
 'removal_reason': None,
 'removed_by': None,
 'removed_by_category': None,
 'report_reasons': None,
 'saved': False,
 'score': 124,
 'secure_media': None,
 'secure_media_embed': {},
 'selftext': 'Every morning I just lay there and hate the fact that I have to go through another day. Sometimes it’s hard for me to go to sleep because I don’t want to wake up and start a whole new miserable day. I feel like a walking zombie, like I’m not even there. I want to feel alive for once. I just feel stuck in my depression.',
 'send_replies': True,
 'spoiler': False,
 'stickied': False,
 'subreddit_id': 't5_2r8rq',
 'subreddit_name_prefixed': 'r/depressed',
 'suggested_sort': None,
 'thumbnail': '',
 'title': 'I hate waking up in the mornings.',
 'total_awards_received': 0,
 'treatment_tags': [],
 'ups': 124,
 'upvote_ratio': 1.0,
 'url': 'https://www.reddit.com/r/depressed/comments/g348ft/i_hate_waking_up_in_the_mornings/',
 'user_reports': [],
 'view_count': None,
 'visited': False,
 'whitelist_status': None,
 'wls': None, 
 'comments':None # My's addition to data structure. Will hold the comments dictionary in this cell.
                 # Otherwise would have to iteratively unpack comments and that won't be consistent across 
                 # each post. ie. one user could have 7 comments and another could have 2. Not to mention 
                 # children of comments. MAYBE WE JUST ADD USERNAME OF COMMENTERS
       }

In [4]:
columns = list(keys.keys()) # all the columns for the data tag

In [14]:
def get_comment_authors(comments_data, authors):
    # recursively get nested commentor ids (used set so there wouldn't be duplicates)
    for k in comments_data.keys():
        # get author of comment
        author = comments_data[k]['data']['author']
        authors.add(author)
        # if comment has child, append
        if comments_data[k]['children']:
            get_comment_authors(comments_data[k]['children'], authors)
    return list(authors)

In [15]:
def json_to_df(path_to_json, columns=columns):
    # list of all files
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    json_data = pd.DataFrame(columns=columns)

    # we need both the json and an index number so use enumerate()
    for index, js in enumerate(json_files):
        with open(os.path.join(path_to_json, js)) as json_file:
            json_text = json.load(json_file)
            # get data 
            values = list(json_text['data'].values())
            # append unique ids of all commentors, add None if DNE
            commentors = get_comment_authors(json_text['comments'], set({}))
            if commentors == []:
                values.append(None)
            else:
                values.append(commentors)
            # push to df by index
            json_data.loc[index] = values
    return json_data

In [16]:
df = json_to_df('../MentalHealth/depressed/posts/')
df.head()

,_comments_by_id,_fetched,all_awardings,allow_live_comments,approved_at_utc,approved_by,archived,author,author_fullname,author_premium,...,treatment_tags,ups,upvote_ratio,url,user_reports,view_count,visited,whitelist_status,wls,comments
0,{},False,[],False,None,None,False,KEY2SO,t2_5lo4i6kc,False,...,[],2,1.0,https://www.reddit.com/r/depressed/comments/h0...,[],None,False,None,None,None
1,{},False,[],False,None,None,False,alleyway_account,t2_87dmucek,False,...,[],2,1.0,https://www.reddit.com/r/depressed/comments/jf...,[],None,False,no_ads,0,"[undercurrents, oh_mypressure, anonymous_helpe..."
2,{},False,[],False,None,None,False,Chilly291,t2_17iwjg,False,...,[],4,1.0,https://www.reddit.com/r/depressed/comments/ir...,[],None,False,no_ads,0,"[None, Patient_Dog4215]"
3,{},False,[],False,None,None,False,corynonymous,t2_6c2phcti,False,...,[],8,1.0,https://www.reddit.com/r/depressed/comments/hx...,[],None,False,None,None,None
4,{},False,[],False,None,None,False,angry_paprika006,t2_68z6e39n,False,...,[],4,1.0,https://www.reddit.com/r/depressed/comments/ig...,[],None,False,no_ads,0,"[jackiemango12335, verlorenundverdammt, Montro..."


In [27]:
print(df.shape)
df.iloc[8]['comments']

(5411, 82)


['ophelie0807',
 'Asdewq123456',
 'Walrusducky',
 'BBB_9',
 '_paradigm_of_sorrow_',
 'jiren420',
 'AnotherTrowaway12',
 'None',
 'xXjustacookieXx',
 'ZaneAura',
 'chickenlishus',
 'undercurrents',
 'prancer4U',
 'chapelboy2006',
 'tihorb',
 'bothp',
 'gottabemanicpanic69',
 'wizardsand',
 'kat123411']

In [28]:
df.to_csv("preprocessed_data/depressed.csv")